In [1]:
%matplotlib inline

# Preprocessing and Spike Sorting Tutorial

# Chapter 1: Importing Recording Data and Metadata

In [2]:
import os
import warnings
import glob
import pickle
import _pickle as cPickle
import imp
import git

/tmp/ipykernel_552169/3524477660.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"

In [4]:
from collections import defaultdict
import time
import json

In [5]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
import scipy.signal
from labellines import labelLine, labelLines
import numpy as np
from scipy.stats import mannwhitneyu

In [6]:
from spectral_connectivity import Multitaper, Connectivity

In [7]:
from probeinterface import get_probe
from probeinterface.plotting import plot_probe, plot_probe_group
from probeinterface import write_prb, read_prb

In [8]:
# Changing the figure size
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

The spikeinterface module by itself import only the spikeinterface.core submodule
which is not useful for end user



In [9]:
import spikeinterface

We need to import one by one different submodules separately (preferred).
There are 5 modules:

- :code:`extractors` : file IO
- :code:`toolkit` : processing toolkit for pre-, post-processing, validation, and automatic curation
- :code:`sorters` : Python wrappers of spike sorters
- :code:`comparison` : comparison of spike sorting output
- :code:`widgets` : visualization



In [10]:
import spikeinterface as si  # import core only
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw

In [11]:
import spikeinterface.core

We can also import all submodules at once with this
  this internally import core+extractors+toolkit+sorters+comparison+widgets+exporters

This is useful for notebooks but this is a more heavy import because internally many more dependency
are imported (scipy/sklearn/networkx/matplotlib/h5py...)



In [12]:
import spikeinterface.full as si

In [13]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (10,6)

# Part 1: Importing Data

## Loading in the Preprocessed LFP 

- Getting the root directory of the Github Repo to base the files off of

In [14]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [15]:
git_root

'/nancy/projects/reward_competition_extention'

- Getting a list of all the electrophysiological recording files
    - **NOTE**: If your recording file does not end with `.rec` or is in a different directory than `./data` then you must change `glob.glob({./path/to/recording_file.rec})` below. Where you replace `{./path/to/recording_file.rec}` with the path to your recording file without the brackets.

In [16]:
lfp_preprocessing_filepath_glob = glob.glob("./proc/*/lfp_preprocessing")

In [17]:
lfp_preprocessing_filepath_glob

['./proc/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged/lfp_preprocessing',
 './proc/20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged/lfp_preprocessing']

## Getting MetaData

In [18]:
subj_1_1_brain_region_to_channel = {"mPFC": "20", "MD": "28", "LH": "29", "BLA": "30", "vHPC": "31"}
subj_1_2_brain_region_to_channel = {"mPFC": "20", "MD": "28", "LH": "29", "BLA": "30", "vHPC": "31"}

In [19]:
tone_df = pd.read_csv("../2023_08_01_tone_timestamp/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec.tone_times.csv")

In [20]:
tone_df

,Unnamed: 0,tone_timestamp,tone_index,rewarded_or_not,competition_winners,condition
0,0,3671086,478042,rewarded,1.1,1.1
1,1,5681112,2488068,rewarded,tie,tie
2,2,8081141,4888097,rewarded,1.2,1.2
3,3,10381173,7188129,rewarded,1.1,1.1
4,4,12181192,8988148,rewarded,1.2,1.2
5,5,13281209,10088165,rewarded,1.1,1.1
6,6,14281218,11088174,rewarded,1.1,1.1
7,7,16181241,12988197,rewarded,1.2,1.2
8,8,17681260,14488216,rewarded,1.1,1.1
9,9,19781285,16588241,rewarded,1.1,1.1


In [21]:
tone_df["lfp_index"] = tone_df["tone_index"] // 20

## Getting LFP Traces

In [22]:
subj_1_1_raw = se.read_spikegadgets("/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec/20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged.rec", stream_id="trodes")
subj_1_2_raw = se.read_spikegadgets("/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec/20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec", stream_id="trodes")

In [23]:
subj_1_1_raw

SpikeGadgetsRecordingExtractor: 32 channels - 1 segments - 20.0kHz - 3418.132s
  file_path: /scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec/20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged.rec

In [24]:
subj_1_1_lfp = spikeinterface.preprocessing.bandpass_filter(subj_1_1_raw, freq_min=0.5, freq_max=300)
subj_1_2_lfp = spikeinterface.preprocessing.bandpass_filter(subj_1_2_raw, freq_min=0.5, freq_max=300)

In [25]:
subj_1_1_lfp = spikeinterface.preprocessing.notch_filter(subj_1_1_lfp, freq=60)
subj_1_2_lfp = spikeinterface.preprocessing.notch_filter(subj_1_2_lfp, freq=60)


In [26]:
resampled_frequency = 1000

In [27]:
subj_1_1_lfp = spikeinterface.preprocessing.resample(subj_1_1_lfp, resample_rate=1000)
subj_1_2_lfp = spikeinterface.preprocessing.resample(subj_1_2_lfp, resample_rate=1000)


# Power Calculation

In [28]:
tone_df["subj_1_1_condition"] = tone_df["condition"].apply(lambda x: "win" if x == "1.1" else ("lose" if x == "1.2" else x))
tone_df["subj_1_2_condition"] = tone_df["condition"].apply(lambda x: "win" if x == "1.2" else ("lose" if x == "1.1" else x))

In [29]:
tone_df

,Unnamed: 0,tone_timestamp,tone_index,rewarded_or_not,competition_winners,condition,lfp_index,subj_1_1_condition,subj_1_2_condition
0,0,3671086,478042,rewarded,1.1,1.1,23902,win,lose
1,1,5681112,2488068,rewarded,tie,tie,124403,tie,tie
2,2,8081141,4888097,rewarded,1.2,1.2,244404,lose,win
3,3,10381173,7188129,rewarded,1.1,1.1,359406,win,lose
4,4,12181192,8988148,rewarded,1.2,1.2,449407,lose,win
5,5,13281209,10088165,rewarded,1.1,1.1,504408,win,lose
6,6,14281218,11088174,rewarded,1.1,1.1,554408,win,lose
7,7,16181241,12988197,rewarded,1.2,1.2,649409,lose,win
8,8,17681260,14488216,rewarded,1.1,1.1,724410,win,lose
9,9,19781285,16588241,rewarded,1.1,1.1,829412,win,lose


In [30]:
tone_df.to_csv("./tone_df_06_20.csv")

In [31]:
def nested_dict():
    return defaultdict(nested_dict)

In [32]:
time_halfbandwidth_product = 1
time_window_duration = 1
time_window_step = 0.1

In [33]:
condition_to_brain_region = nested_dict()

In [34]:
for index, row in tone_df.iterrows():
    for brain_region, channel_id in subj_1_1_brain_region_to_channel.items():
        try:
            current_trace = subj_1_1_lfp.get_traces(channel_ids=[channel_id], start_frame=row["lfp_index"], end_frame=row["lfp_index"] + 1000 * 10)
            # condition_to_brain_region[row["subj_1_1_condition"]][brain_region]["traces"] = current_trace
            
                    # Compute multitaper spectral estimate
            m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=current_trace,
                           sampling_frequency=resampled_frequency)
            
            # Sets up computing connectivity measures/power from multitaper spectral estimate
            c = Connectivity.from_multitaper(m)
                
            if condition_to_brain_region[row["subj_1_1_condition"]][brain_region]:
                condition_to_brain_region[row["subj_1_1_condition"]][brain_region].append(c.power().squeeze())
            else:
                condition_to_brain_region[row["subj_1_1_condition"]][brain_region] = [c.power().squeeze()]
        except Exception as e: 
            print(e)
            print("issue with {} {} {}".format(brain_region, channel_id, index))

The length of the input vector x must be greater than padlen, which is 33.
issue with mPFC 20 40
The length of the input vector x must be greater than padlen, which is 33.
issue with MD 28 40
The length of the input vector x must be greater than padlen, which is 33.
issue with LH 29 40
The length of the input vector x must be greater than padlen, which is 33.
issue with BLA 30 40
The length of the input vector x must be greater than padlen, which is 33.
issue with vHPC 31 40


In [35]:
for index, row in tone_df.iterrows():
    for brain_region, channel_id in subj_1_2_brain_region_to_channel.items():
        try:
            current_trace = subj_1_2_lfp.get_traces(channel_ids=[channel_id], start_frame=row["lfp_index"], end_frame=row["lfp_index"] + 1000 * 10)
            # condition_to_brain_region[row["subj_1_2_condition"]][brain_region]["traces"] = current_trace
            
                    # Compute multitaper spectral estimate
            m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=current_trace,
                           sampling_frequency=resampled_frequency)
            
            # Sets up computing connectivity measures/power from multitaper spectral estimate
            c = Connectivity.from_multitaper(m)
                
            if condition_to_brain_region[row["subj_1_2_condition"]][brain_region]:
                condition_to_brain_region[row["subj_1_2_condition"]][brain_region].append(c.power().squeeze())
            else:
                condition_to_brain_region[row["subj_1_2_condition"]][brain_region] = [c.power().squeeze()]
        except Exception as e: 
            print(e)
            print("issue with {} {} {}".format(brain_region, channel_id, index))

The length of the input vector x must be greater than padlen, which is 33.
issue with mPFC 20 40
The length of the input vector x must be greater than padlen, which is 33.
issue with MD 28 40
The length of the input vector x must be greater than padlen, which is 33.
issue with LH 29 40
The length of the input vector x must be greater than padlen, which is 33.
issue with BLA 30 40
The length of the input vector x must be greater than padlen, which is 33.
issue with vHPC 31 40


In [36]:
len(condition_to_brain_region["win"]["mPFC"])

19

In [37]:
c.frequencies

array([0.000e+00, 1.000e-01, 2.000e-01, ..., 4.997e+02, 4.998e+02,
       4.999e+02])

In [38]:
losing_omission_freq_index = np.where((6 <= c.frequencies) & (12 >= c.frequencies))[0]

In [39]:
losing_omission_freq_index

array([ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
        86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120])

In [40]:
winning_rewarded_freq_index = np.where((2 <= c.frequencies) & (6 >= c.frequencies))[0]

In [41]:
winning_rewarded_freq_index

array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60])

In [42]:
freq_index_6_to_12 = np.where((6 <= c.frequencies) & (12 >= c.frequencies))[0]

In [43]:
freq_index_6_to_12

array([ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,
        86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 113, 114, 115, 116, 117, 118, 119, 120])

In [44]:
freq_index_2_to_6 = np.where((2 <= c.frequencies) & (6 >= c.frequencies))[0]

In [45]:
freq_index_2_to_6

array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60])

In [46]:
freq_to_index = {"freq_6_to_12": freq_index_6_to_12, "freq_2_to_6": freq_index_2_to_6}

In [47]:
average_power_dict = nested_dict()
for freq, index in freq_to_index.items():
    print(freq)
    for condition, all_regions in condition_to_brain_region.items():   
        print(condition)
        for brain_region, value in all_regions.items():
            print(brain_region)
            current_all_power = [powers[index] for powers in value]
            per_trial_average_power = [np.mean(powers) for powers in current_all_power]
            average_power_dict[condition][brain_region][freq]["per_trial_average_power"] = per_trial_average_power


freq_6_to_12
win
mPFC
MD
LH
BLA
vHPC
tie
mPFC
MD
LH
BLA
vHPC
lose
mPFC
MD
LH
BLA
vHPC
rewarded
mPFC
MD
LH
BLA
vHPC
omission
mPFC
MD
LH
BLA
vHPC
freq_2_to_6
win
mPFC
MD
LH
BLA
vHPC
tie
mPFC
MD
LH
BLA
vHPC
lose
mPFC
MD
LH
BLA
vHPC
rewarded
mPFC
MD
LH
BLA
vHPC
omission
mPFC
MD
LH
BLA
vHPC


In [48]:
condition

'omission'

In [49]:
average_power_dict["omission"][brain_region][freq]["per_trial_average_power"]

[12067.892364557461,
 13116.706714481266,
 13839.496737192723,
 14180.112305669012,
 45961.320258485415,
 36955.28830399298,
 31404.102852536296,
 38444.91938649119]

In [50]:
# Convert the triple-nested dictionary to a DataFrame
average_power_df = pd.concat({(i, j): pd.DataFrame(k) for i, jk in average_power_dict.items() for j, k in jk.items()}, axis=0)

# Reset the index and rename the columns
average_power_df = average_power_df.reset_index()

In [60]:
average_power_df = average_power_df.rename(columns={"level_0": "condition", "level_1": "brain_region"})

In [66]:
average_power_df

,condition,brain_region,level_2,freq_6_to_12,freq_2_to_6
0,win,mPFC,per_trial_average_power,"[1570.0376363392365, 1545.6203273957892, 1536....","[312.0248082592546, 585.2948161301464, 792.004..."
1,win,MD,per_trial_average_power,"[26668.65603155145, 18113.717676006036, 23119....","[5356.85641890281, 8033.724644062721, 7116.970..."
2,win,LH,per_trial_average_power,"[18572.57769692026, 11848.204933338538, 16038....","[4128.380578123925, 5993.045083266717, 6012.15..."
3,win,BLA,per_trial_average_power,"[21200.784142423785, 11339.163715952116, 16217...","[5626.786050137386, 7420.06225506192, 7303.107..."
4,win,vHPC,per_trial_average_power,"[16454.40754678111, 10511.411066519513, 14325....","[12523.011786282228, 15723.718562698892, 20390..."
5,tie,mPFC,per_trial_average_power,"[1317.7988077199907, 7721.669275346898]","[759.9575294132649, 9529.217179366033]"
6,tie,MD,per_trial_average_power,"[18872.78936115108, 13044.70196009008]","[10118.987593744114, 7208.501955823576]"
7,tie,LH,per_trial_average_power,"[13116.67601358231, 5944.502323963041]","[6492.278223985535, 4112.857734149657]"
8,tie,BLA,per_trial_average_power,"[14291.385209133614, 5871.912576138091]","[9837.187520879455, 7037.302012891251]"
9,tie,vHPC,per_trial_average_power,"[10750.332662620584, 14171.627519876653]","[14598.516269726528, 18390.534074612715]"


In [61]:
average_power_df.to_csv("./average_power_06_20.csv")

In [75]:
import ast

In [77]:
other_average_power_df = pd.read_csv("./average_power_06_18.csv", index_col=0)

In [78]:

other_average_power_df["freq_2_to_6"] = othe_average_power_df["freq_2_to_6"].apply(lambda x: ast.literal_eval(x))
other_average_power_df["freq_6_to_12"] = othe_average_power_df["freq_6_to_12"].apply(lambda x: ast.literal_eval(x))

In [79]:
other_average_power_df

,condition,brain_region,level_2,freq_6_to_12,freq_2_to_6
0,lose,mPFC,per_trial_average_power,"[3659.1399851404585, 5104.409429016465, 3360.0...","[2698.6030563663735, 1482.9162669010263, 2081...."
1,lose,MD,per_trial_average_power,"[24476.01909643453, 23821.80504122089, 19633.8...","[10801.4646778569, 5497.79554862735, 8507.6662..."
2,lose,LH,per_trial_average_power,"[18546.43353100673, 17037.994017704073, 13974....","[7020.490913238811, 5494.045735171809, 6685.98..."
3,lose,BLA,per_trial_average_power,"[17607.89435654146, 16276.936113462909, 15529....","[8745.73348140386, 5166.602083543791, 7582.821..."
4,lose,vHPC,per_trial_average_power,"[11143.55033263804, 11795.783796087177, 14414....","[11363.120013025402, 9545.616496617315, 17621...."
5,win,mPFC,per_trial_average_power,"[2458.5760188265062, 3385.4492798785677, 3568....","[1481.3047546944538, 1115.775720650603, 2791.7..."
6,win,MD,per_trial_average_power,"[18422.63638618245, 18803.0478759335, 18856.53...","[5374.440345054873, 4957.737593966543, 9693.80..."
7,win,LH,per_trial_average_power,"[12674.814488993212, 12558.265596047764, 13408...","[4491.348805117406, 3975.3310007746836, 7263.1..."
8,win,BLA,per_trial_average_power,"[14809.768609197574, 13179.27476594031, 14580....","[5401.508478035267, 3737.999005097903, 6859.28..."
9,win,vHPC,per_trial_average_power,"[8253.389251279476, 10852.35083517253, 10759.3...","[11732.557298220772, 9238.234521903034, 11066...."


In [82]:
combined_average_power_df = pd.merge(average_power_df, other_average_power_df, on=['condition', 'brain_region'])

In [83]:
combined_average_power_df

,condition,brain_region,level_2_x,freq_6_to_12_x,freq_2_to_6_x,level_2_y,freq_6_to_12_y,freq_2_to_6_y
0,win,mPFC,per_trial_average_power,"[1570.0376363392365, 1545.6203273957892, 1536....","[312.0248082592546, 585.2948161301464, 792.004...",per_trial_average_power,"[2458.5760188265062, 3385.4492798785677, 3568....","[1481.3047546944538, 1115.775720650603, 2791.7..."
1,win,MD,per_trial_average_power,"[26668.65603155145, 18113.717676006036, 23119....","[5356.85641890281, 8033.724644062721, 7116.970...",per_trial_average_power,"[18422.63638618245, 18803.0478759335, 18856.53...","[5374.440345054873, 4957.737593966543, 9693.80..."
2,win,LH,per_trial_average_power,"[18572.57769692026, 11848.204933338538, 16038....","[4128.380578123925, 5993.045083266717, 6012.15...",per_trial_average_power,"[12674.814488993212, 12558.265596047764, 13408...","[4491.348805117406, 3975.3310007746836, 7263.1..."
3,win,BLA,per_trial_average_power,"[21200.784142423785, 11339.163715952116, 16217...","[5626.786050137386, 7420.06225506192, 7303.107...",per_trial_average_power,"[14809.768609197574, 13179.27476594031, 14580....","[5401.508478035267, 3737.999005097903, 6859.28..."
4,win,vHPC,per_trial_average_power,"[16454.40754678111, 10511.411066519513, 14325....","[12523.011786282228, 15723.718562698892, 20390...",per_trial_average_power,"[8253.389251279476, 10852.35083517253, 10759.3...","[11732.557298220772, 9238.234521903034, 11066...."
5,lose,mPFC,per_trial_average_power,"[1760.2066132746377, 1331.0472385618216, 1648....","[434.3774350182326, 738.2448492152461, 941.250...",per_trial_average_power,"[3659.1399851404585, 5104.409429016465, 3360.0...","[2698.6030563663735, 1482.9162669010263, 2081...."
6,lose,MD,per_trial_average_power,"[21443.665819317106, 16300.558452605947, 18566...","[3663.354545349904, 6488.525569301361, 5682.03...",per_trial_average_power,"[24476.01909643453, 23821.80504122089, 19633.8...","[10801.4646778569, 5497.79554862735, 8507.6662..."
7,lose,LH,per_trial_average_power,"[15727.988454162429, 10419.702868888171, 13005...","[3320.01877465733, 4606.892547760393, 3991.328...",per_trial_average_power,"[18546.43353100673, 17037.994017704073, 13974....","[7020.490913238811, 5494.045735171809, 6685.98..."
8,lose,BLA,per_trial_average_power,"[15172.669655086867, 10933.391643153056, 13573...","[3751.768411891569, 5440.622319399708, 5782.56...",per_trial_average_power,"[17607.89435654146, 16276.936113462909, 15529....","[8745.73348140386, 5166.602083543791, 7582.821..."
9,lose,vHPC,per_trial_average_power,"[14022.395795540537, 11076.7903614308, 13068.3...","[10735.90909895402, 10143.236144653665, 10396....",per_trial_average_power,"[11143.55033263804, 11795.783796087177, 14414....","[11363.120013025402, 9545.616496617315, 17621...."


In [85]:
combined_average_power_df["freq_6_to_12"] = combined_average_power_df["freq_6_to_12_x"] + combined_average_power_df["freq_6_to_12_y"]
combined_average_power_df["freq_2_to_6"] = combined_average_power_df["freq_2_to_6_x"] + combined_average_power_df["freq_2_to_6_x"]

In [110]:
for index in combined_average_power_df["freq_6_to_12"]:
    print(len(index))

39
39
39
39
39
39
39
39
39
39
62
62
62
62
62
14
14
14
14
14


In [ ]:
condition_to_brain_region

In [88]:
freq

'freq_6_to_12'

In [92]:
lose_df[freq].iloc[0]

[1760.2066132746377,
 1331.0472385618216,
 1648.1874785980278,
 1812.331221763868,
 1717.7166614841271,
 1137.9778004257785,
 7760.819660610264,
 4961.156734194947,
 4392.147423837756,
 5610.032681410756,
 7075.129326405369,
 4033.9086174387958,
 8825.172838851226,
 6319.321479144285,
 3098.9202590896643,
 8783.74683882759,
 4966.5344434785075,
 4074.793740568948,
 4048.620695650724,
 3659.1399851404585,
 5104.409429016465,
 3360.0137457847636,
 3646.9685636642225,
 3252.482908721951,
 4092.532552327542,
 898.6585510819805,
 769.8192941609287,
 1293.6409504311923,
 809.2654056291736,
 681.8641478372251,
 881.7107272604482,
 1137.3792412422306,
 1101.1376795196593,
 870.5894583200827,
 862.9059634842232,
 913.9822375612122,
 893.8063173685372,
 866.22924334755,
 1078.479337853974]

In [93]:
brain_region_and_condition_to_pvalue = nested_dict()
for freq, index in freq_to_index.items():
    for brain_region in subj_1_1_brain_region_to_channel.keys():
        print(brain_region)

        lose_df = combined_average_power_df[(combined_average_power_df["condition"]=="lose") & (combined_average_power_df["brain_region"] == brain_region)]
        win_df = combined_average_power_df[(combined_average_power_df["condition"]=="win") & (combined_average_power_df["brain_region"] == brain_region)]
        rewarded_df = combined_average_power_df[(combined_average_power_df["condition"]=="rewarded") & (combined_average_power_df["brain_region"] == brain_region)]
        omission_df = combined_average_power_df[(combined_average_power_df["condition"]=="omission") & (combined_average_power_df["brain_region"] == brain_region)]
        
        u_statistic, p_value = mannwhitneyu(lose_df[freq].iloc[0], omission_df[freq].iloc[0])
        brain_region_and_condition_to_pvalue[brain_region]["losing_vs_omission"]["u_statistic"][freq] = u_statistic
        brain_region_and_condition_to_pvalue[brain_region]["losing_vs_omission"]["p_value"][freq] = p_value
    
        
        u_statistic, p_value = mannwhitneyu(win_df[freq].iloc[0], rewarded_df[freq].iloc[0])
        brain_region_and_condition_to_pvalue[brain_region]["winning_vs_rewarded"]["u_statistic"][freq] = u_statistic
        brain_region_and_condition_to_pvalue[brain_region]["winning_vs_rewarded"]["p_value"][freq] = p_value

mPFC
MD
LH
BLA
vHPC
mPFC
MD
LH
BLA
vHPC


In [94]:
brain_region_and_condition_to_pvalue

defaultdict(<function __main__.nested_dict()>,
            {'mPFC': defaultdict(<function __main__.nested_dict()>,
                         {'losing_vs_omission': defaultdict(<function __main__.nested_dict()>,
                                      {'u_statistic': defaultdict(<function __main__.nested_dict()>,
                                                   {'freq_6_to_12': 323.0,
                                                    'freq_2_to_6': 284.0}),
                                       'p_value': defaultdict(<function __main__.nested_dict()>,
                                                   {'freq_6_to_12': 0.3179761769815497,
                                                    'freq_2_to_6': 0.7116908682615839})}),
                          'winning_vs_rewarded': defaultdict(<function __main__.nested_dict()>,
                                      {'u_statistic': defaultdict(<function __main__.nested_dict()>,
                                                   {'freq_6_to_12

In [95]:
# Convert the triple-nested dictionary to a DataFrame
lfp_wilconxon_df = pd.concat({(i, j): pd.DataFrame(k) for i, jk in brain_region_and_condition_to_pvalue.items() for j, k in jk.items()}, axis=0)

# Reset the index and rename the columns
lfp_wilconxon_df = lfp_wilconxon_df.reset_index()

In [96]:
lfp_wilconxon_df = lfp_wilconxon_df.rename(columns={"level_0": "brain_region", "level_1": "condition", "level_2": "frequency"})

In [97]:
lfp_wilconxon_df = lfp_wilconxon_df.sort_values(["brain_region", "condition", "frequency"]).reset_index(drop=True)[["brain_region", "condition", "frequency", "p_value", "u_statistic"]]

In [98]:
lfp_wilconxon_df

,brain_region,condition,frequency,p_value,u_statistic
0,BLA,losing_vs_omission,freq_2_to_6,0.200779,236.0
1,BLA,losing_vs_omission,freq_6_to_12,0.229996,333.0
2,BLA,winning_vs_rewarded,freq_2_to_6,1.000000,1216.0
3,BLA,winning_vs_rewarded,freq_6_to_12,0.003281,1631.0
4,LH,losing_vs_omission,freq_2_to_6,0.000061,92.0
5,LH,losing_vs_omission,freq_6_to_12,0.113267,352.0
6,LH,winning_vs_rewarded,freq_2_to_6,0.294300,1064.0
7,LH,winning_vs_rewarded,freq_6_to_12,0.007019,1596.0
8,MD,losing_vs_omission,freq_2_to_6,0.049803,200.0
9,MD,losing_vs_omission,freq_6_to_12,0.246041,331.0


In [102]:
lfp_wilconxon_df.to_csv("./combined_lfp_wilcoxon.csv")

# Comparing Spikes

In [ ]:
raise ValueError()

In [ ]:
subj_1_4_lfp.get_traces(channel_ids=start_frame=100, end_frame=200)

In [ ]:
sorted(list(subj_1_1_brain_region_to_channel.values()))

In [ ]:
sorted(list(subj_1_4_brain_region_to_channel.values()))

In [ ]:
subj_1_1_lfp

In [ ]:
subj_1_4_lfp

In [ ]:
subj_1_1_traces = subj_1_1_lfp.get_traces(channel_ids=subj_1_1_lfp.get_channel_ids()[[20, 28, 29, 30, 31]])

In [ ]:
subj_1_4_traces = subj_1_4_lfp.get_traces(channel_ids=subj_1_4_lfp.get_channel_ids()[[2, 28, 29, 30, 31]])

In [ ]:
subj_1_4_traces

In [ ]:
subj_1_1_traces.shape

In [ ]:
# Compute multitaper spectral estimate
subj_1_1_m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=subj_1_1_traces,
               sampling_frequency=resampled_frequency)

# Sets up computing connectivity measures/power from multitaper spectral estimate
c = Connectivity.from_multitaper(m)

In [ ]:
### Reading in the recording
trodes_recording = se.read_spikegadgets(recording_path, stream_id="trodes")

### Preprocessing the recording
preprocessing_step_dict = defaultdict(dict)
preprocessing_step_dict["recording_raw"] = trodes_recording

recording_intermediate = copy.copy(trodes_recording)
# Bandpass Filtering, keeping all the points that are within a certain frequency range
recording_bandpass = spikeinterface.preprocessing.bandpass_filter(recording_intermediate, freq_min=0.5, freq_max=300)
recording_intermediate = spikeinterface.preprocessing.bandpass_filter(recording_intermediate, freq_min=0.5, freq_max=300)
preprocessing_step_dict["recording_bandpass"] = recording_bandpass
print(recording_bandpass)

# Notch Filtering, keeping all the points that are within a certain frequency range
recording_notch = spikeinterface.preprocessing.notch_filter(recording_intermediate, freq=60)
recording_intermediate = spikeinterface.preprocessing.notch_filter(recording_intermediate, freq=60)
preprocessing_step_dict["recording_notch"] = recording_notch
print(recording_bandpass)

# We are not going to run the resampling step because it causes issues with saving to file
# Resampling
recording_resample = spikeinterface.preprocessing.resample(recording_intermediate, resample_rate=resampled_frequency)
preprocessing_step_dict["recording_resample"] = recording_resample
print(recording_resample)

In [ ]:
raise ValueError()

```
MD EIB: 0(1 on Nancy notebook) Spike Gadgets/Spike Interface: 0
vHPC EIB: 16(17 on Nancy notebook) Spike Gadgets/Spike Interface: 31
BLA EIB: 17(18 on Nancy notebook) Spike Gadgets/Spike Interface: 30
LH EIB: 18(19 on Nancy notebook) Spike Gadgets/Spike Interface: 29
```

In [ ]:
brain_region_to_channel = {"mPFC": "21", "MD": "16", "vHPC": "15", "BLA": "14", "LH": "13"}

In [ ]:
brain_region_to_channel = {k: v for k, v in sorted(brain_region_to_channel.items(), key=lambda item: item[1])}

In [ ]:
brain_region_to_channel.keys()

In [ ]:
brain_region_to_channel.values()

In [ ]:
channel_ids = brain_region_to_channel.values()
time_range=(1000, 1005)
resampled_frequency = 1000

In [ ]:
time_halfbandwidth_product=10

### Brain Region to Channel

```
- All Trodes
    - 21, 31, 15, 14, 13
- mPFC:
    - 21 (0 to 31 on Spike interface)
    - 22(1 to 32 on Trodes)
- MD
    - 1 (EIB)
        - Maybe actually 16
    - 31(0 to 31 Spike interface)
        - Maybe actually 16
    - 32(1 to 32 Trodes)
- vHPC
    - 17(EIB)
    - 15(0 to 31 on Spike interface)
    - 16(1 to 32 Trodes)
- BLA
    - 18(BLA)
    - 14(0 to 31 Spike interface)
    - 15(1 to 32 Trodes)
- LH 18
    - 19(EIB)
    - 13(0 to 31 Spike interface)
    - 14(1 to 32 Trodes)
```

In [ ]:
resampled_frequency = 1000

In [ ]:
recording_intermediate.channel_ids

In [ ]:
for preprocessed_lfp_path in lfp_preprocessing_filepath_glob:
    print(preprocessed_lfp_path)
    recording_intermediate = spikeinterface.core.load_extractor(file_or_folder_or_dict=preprocessed_lfp_path)
    recording_basename = os.path.basename(os.path.dirname(preprocessed_lfp_path))
    parent_output_directory = os.path.dirname(preprocessed_lfp_path)

    
    # Resampling
    recording_intermediate = spikeinterface.preprocessing.resample(recording_intermediate, resample_rate=resampled_frequency) 
#     sw.plot_timeseries(recording_intermediate, time_range=time_range, channel_ids=channel_ids, show_channel_ids=True)

    # Getting the traces
    
    traces = recording_intermediate.get_traces(start_frame=1100*1000, end_frame=1160*1000)
    
    # Compute multitaper spectral estimate
    m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=traces,
                   sampling_frequency=resampled_frequency)
    
    # Sets up computing connectivity measures/power from multitaper spectral estimate
    c = Connectivity.from_multitaper(m)
    
    for num in range(8):
        channels = [int(num) for num in recording_intermediate.channel_ids[num*4: num*4+4]]
        
    
    
        fig, ax = plt.subplots()
        ax.set_xlabel("Frequency")
        ax.set_ylabel("Power")
        ax.plot(c.frequencies, c.power().squeeze()[:,channels], label=channels)
        ax.set_xlim(1,12)
        ax.legend(ncol=4)
        labelLines(ax.get_lines(), zorder=2.5)
        plt.title(recording_basename)
        os.makedirs('./proc/{}'.format(recording_basename), exist_ok=True)
        plt.savefig('./proc/{}/{}_ch_{}.png'.format(recording_basename, recording_basename, "-".join([str(ch) for ch in channels])))
        plt.show()


In [ ]:
for preprocessed_lfp_path in lfp_preprocessing_filepath_glob:
    print(preprocessed_lfp_path)
    recording_intermediate = spikeinterface.core.load_extractor(file_or_folder_or_dict=preprocessed_lfp_path)
    recording_basename = os.path.basename(os.path.dirname(preprocessed_lfp_path))
    parent_output_directory = os.path.dirname(preprocessed_lfp_path)

    
    # Resampling
    recording_intermediate = spikeinterface.preprocessing.resample(recording_intermediate, resample_rate=resampled_frequency) 
#     sw.plot_timeseries(recording_intermediate, time_range=time_range, channel_ids=channel_ids, show_channel_ids=True)

    # Getting the traces
    
    traces = recording_intermediate.get_traces(start_frame=1000*1000, end_frame=1010*1000)
    
    # Compute multitaper spectral estimate
    m = Multitaper(time_halfbandwidth_product=time_halfbandwidth_product, time_series=traces,
                   sampling_frequency=resampled_frequency)
    
    # Sets up computing connectivity measures/power from multitaper spectral estimate
    c = Connectivity.from_multitaper(m)
    
    for num in range(8):
        channels = [int(num) for num in recording_intermediate.channel_ids[num*4: num*4+4]]
        
    
    
        fig, ax = plt.subplots()
        ax.set_xlabel("Frequency")
        ax.set_ylabel("Power")
        ax.plot(c.frequencies, c.power().squeeze()[:,channels], label=channels)
        ax.set_xlim(1,12)
        ax.set_ylim(1,2000)

        ax.legend(ncol=4)
        labelLines(ax.get_lines(), zorder=2.5)
        plt.title(recording_basename)
        plt.show()


### Get all the traces

In [ ]:
raise ValueError()

In [ ]:
recording_root_to_everything = defaultdict(dict)

In [ ]:
resampled_frequency = 1000

In [ ]:
np_issue_files = []
for preprocessed_lfp_path in lfp_preprocessing_filepath_glob:
    try:
        print(preprocessed_lfp_path)
        recording_intermediate = spikeinterface.core.load_extractor(file_or_folder_or_dict=preprocessed_lfp_path)
        recording_basename = os.path.basename(os.path.dirname(preprocessed_lfp_path))
        parent_output_directory = os.path.dirname(preprocessed_lfp_path)

        # Resampling
        recording_intermediate = spikeinterface.preprocessing.resample(recording_intermediate, resample_rate=resampled_frequency) 

        # Getting the traces
        
        traces = recording_intermediate.get_traces(start_frame=0, end_frame=recording_intermediate.get_num_samples()-1)
        # Saving the trace to a Numpy file
        traces_filename = "{}.traces.npy".format(recording_basename)
        np.save(file=os.path.join(parent_output_directory, traces_filename), arr=traces)
    except:
        np_issue_files.append(preprocessed_lfp_path)